# JPMorgan Quantitative Research
## Task 4: Bucket FICO scores

In [30]:
# import relevant packages
import numpy as np
import pandas as pd
from google.colab import drive

# read and check the file
drive.mount('/content/drive', force_remount = True)
file_path = '/content/drive/MyDrive/Forage/Task 3 and 4_Loan_Data.csv'
loan_data = pd.read_csv(file_path)
loan_data.head()

Mounted at /content/drive


,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0


In [31]:
loan_data.describe()

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,4.974577e+06,1.461200,4159.677034,8718.916797,70039.901401,4.552800,637.557700,0.185100
std,2.293890e+06,1.743846,1421.399078,6627.164762,20072.214143,1.566862,60.657906,0.388398
min,1.000324e+06,0.000000,46.783973,31.652732,1000.000000,0.000000,408.000000,0.000000
25%,2.977661e+06,0.000000,3154.235371,4199.836020,56539.867903,3.000000,597.000000,0.000000
50%,4.989502e+06,1.000000,4052.377228,6732.407217,70085.826330,5.000000,638.000000,0.000000
75%,6.967210e+06,2.000000,5052.898103,11272.263740,83429.166133,6.000000,679.000000,0.000000
max,8.999789e+06,5.000000,10750.677810,43688.784100,148412.180500,10.000000,850.000000,1.000000


In [32]:
# define a function that maps FICO scores into simple ratings with lower ratings indicating better scores
def fico_buckets(df = loan_data, fico_score_column = 'fico_score', default_column = 'default', buckets = 5):
  # simplify the dataframe
  new_df = df.groupby(fico_score_column)[default_column].agg(count = 'count', defaults = 'sum')
  fico_score = new_df.index.to_numpy()
  n = new_df['count'].to_numpy()
  k = new_df['defaults'].to_numpy()

  # check that the number of buckets work
  m = len(fico_score)
  if buckets > m:
    buckets = m
    print('The number of buckets selected is switched to {}.'.format(buckets))

  # define the log-likelihood method
  def log_likelihood(i, j):
    n_bucket = sum(n[i: j + 1])
    k_bucket = sum(k[i: j + 1])
    if n_bucket > 0:
      p_bucket = k_bucket / n_bucket
    else:
      return - np.inf
    eps = 1e-10
    p_bucket = np.clip(p_bucket, eps, 1 - eps)
    LL = k_bucket * np.log(p_bucket) + (n_bucket - k_bucket) * np.log(1 - p_bucket)
    return LL

  # initialise the dynamic programming table
  dp = np.full((m, buckets + 1), - np.inf)
  prev = np.full((m, buckets + 1), -1)

  # dynamic programming table for 1 bucket
  for j in range(m):
    dp[j, 1] = log_likelihood(0, j)

  # dynamic programming table for 2 buckets onwards
  for b in range(2, buckets + 1):
    for j in range(m):
      for i in range(j):
        candidate = dp[i, b - 1] + log_likelihood(i + 1, j)
        if candidate > dp[j, b]:
          dp[j, b] = candidate
          prev[j, b] = i

  # get the boundaries of the buckets
  boundaries = []
  j = m - 1
  b = buckets
  while b > 1:
    i = int(prev[j, b])
    boundaries.append(int(fico_score[i]))
    j = i
    b -= 1
  boundaries = sorted(boundaries)

  # define a rating map
  def rating_map(score, boundaries):
    for a, boundary in enumerate(boundaries):
      if score <= boundary:
        return buckets - a
    return 1

  # get the new ratings for each FICO score
  df_copy = df.copy()
  df_copy['rating'] = df_copy[fico_score_column].map(lambda x: rating_map(x, boundaries))
  return boundaries, df_copy.head()

while True:
  try:
    num_buckets = int(input('Enter how many buckets you would like to sort the FICO scores into: '))
    if num_buckets >= 1:
      break
    else:
      print('This is not a valid number.')
  except ValueError:
    print('This is not a valid number.')

boundaries, table = fico_buckets(loan_data, 'fico_score', 'default', num_buckets)
print('Here are the boundaries that best summarise the data:\n', boundaries)
print('Here are the ratings for each FICO score:')
table

Enter how many buckets you would like to sort the FICO scores into: 7
Here are the boundaries that best summarise the data:
 [520, 552, 580, 611, 649, 719]
Here are the ratings for each FICO score:


,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default,rating
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0,4
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1,5
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0,4
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0,3
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0,3
